# Cora with pytorch geometric

In [18]:
import torch
from torch_geometric.datasets import Planetoid

## Load data

In [19]:
dataset = Planetoid(root="data", name = "Cora")

Processing...
Done!


In [20]:
print(dataset)
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)
print(dataset.num_edge_features)

Cora()
1
7
1433
0


In [21]:
dataset.data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [22]:
dataset.data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [23]:
dataset.data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [24]:
dataset.data.x.shape

torch.Size([2708, 1433])

In [26]:
dataset.data.y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [27]:
dataset.data.x[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

## Modeling

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [36]:
data = dataset[0]

In [37]:
class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.conv2 = GCNConv(32, 16)
        self.conv3 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)

        return F.log_softmax(x, dim = 1)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, data = GCN().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2, weight_decay=5e-4)

In [42]:
model.train()

for epoch in range(250):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f"Loss: {loss.item()}")

Loss: 1.9410877227783203
Loss: 1.8530105352401733
Loss: 1.749307632446289
Loss: 1.5974714756011963
Loss: 1.4630048274993896
Loss: 1.313302993774414
Loss: 1.1401771306991577
Loss: 0.9885607957839966
Loss: 0.8730276226997375
Loss: 0.7507880330085754
Loss: 0.6332305073738098
Loss: 0.6043334603309631
Loss: 0.5287462472915649
Loss: 0.41219156980514526
Loss: 0.42277616262435913
Loss: 0.3233049511909485
Loss: 0.30313315987586975
Loss: 0.2742003798484802
Loss: 0.21048885583877563
Loss: 0.1837950348854065
Loss: 0.20244139432907104
Loss: 0.13810287415981293
Loss: 0.11276200413703918
Loss: 0.09405592083930969
Loss: 0.12875646352767944
Loss: 0.15877306461334229
Loss: 0.08602245151996613
Loss: 0.06638729572296143
Loss: 0.18255291879177094
Loss: 0.07169251888990402
Loss: 0.11679030954837799
Loss: 0.0990569218993187
Loss: 0.08418361097574234
Loss: 0.10605935007333755
Loss: 0.08982821553945541
Loss: 0.07161252945661545
Loss: 0.09290781617164612
Loss: 0.0444791242480278
Loss: 0.04541029408574104
Loss: 